In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../datasets/abdallah_books_with_descriptions.csv")

In [3]:
# Each book can have a subtitle -> Normalize data

df['subtitle'] = df['subtitle'].fillna('')
titles = df["title"]
subtitles = df["subtitle"]
complete_titles = []

for i in range(0,len(titles)):
    if subtitles[i]:
        complete_titles.append(titles[i] + ' - ' + str(subtitles[i]))
    else: 
        complete_titles.append(titles[i])

df['title'] = complete_titles

# Drop the subtitle column
df.drop(columns=['subtitle'], inplace=True)
df.head(3)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web - A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0


In [4]:
# Problem -> Fiction categories too split -> too many sub genres
# Get all fiction categories into one list
li = []
df['categories'] = df['categories'].fillna('')
for x in df['categories']:
    if "fiction" in x.lower() and x.lower() not in li:
        li.append(x.lower())

all_fictions = li
len(all_fictions)

33

In [5]:
df['categories'] = df['categories'].str.lower()
df_fiction_union = df
df_fiction_union['categories'] = df['categories'].replace(all_fictions, 'Fiction')
# df_fiction_union['categories'].head(50)
df_fiction_union['categories'].value_counts()[35:50]

categories
sports & recreation       17
foreign language study    12
true crime                12
photography               11
law                       11
reference                 11
architecture              10
authors, american         10
crafts & hobbies           9
england                    9
fantasy                    9
pets                       8
short stories              8
americans                  8
games & activities         8
Name: count, dtype: int64

In [6]:
category_counts = df_fiction_union['categories'].value_counts()
threshold = 2

# Filter categories with counts less than the threshold -> What to do with them?
rare_categories = category_counts[category_counts < threshold]
rare_categories[:10]

categories
escapes                                1
bedtime                                1
cosmetics industry                     1
reducing diets                         1
partition of decedents' estates        1
explorers                              1
confianza en sí mismo                  1
archaeological expeditions             1
castle rock (me. : imaginary place)    1
dallas, eve (fictitious character)     1
Name: count, dtype: int64

In [7]:
df_fiction_union.head(3)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web - A Novel,Charles Osborne;Agatha Christie,detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,Stephen R. Donaldson,Fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0


In [8]:
temp_df = df_fiction_union[['title', 'authors', 'categories', 'description', 'average_rating', 'ratings_count']]
# temp_df.to_csv("./datasets/Processed_data_temp.csv")
temp_df.head(3)

,title,authors,categories,description,average_rating,ratings_count
0,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...,3.85,361.0
1,Spider's Web - A Novel,Charles Osborne;Agatha Christie,detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,3.83,5164.0
2,The One Tree,Stephen R. Donaldson,Fiction,Volume Two of Stephen Donaldson's acclaimed se...,3.97,172.0


In [9]:
# Some are tagged with "ficticious characters" => make them simply "fiction"
li = []
temp_df['categories'] = temp_df['categories'].fillna('')
for x in temp_df['categories']:
    if "fict" in x.lower() and x.lower() not in li:
        li.append(x.lower())
li = li[1:]

C:\Users\Raul\AppData\Local\Temp\ipykernel_8204\267572280.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['categories'] = temp_df['categories'].fillna('')


In [10]:
# mask = temp_df['categories'].apply(lambda x: any(cat.lower() in x.lower() for cat in li))

# # Apply the mask to filter the DataFrame
# filtered_df = df[mask]
# filtered_df[['title','categories']]

df_fiction_union = temp_df
df_fiction_union['categories'] = df['categories'].replace(li, 'Fiction')

C:\Users\Raul\AppData\Local\Temp\ipykernel_8204\2893261262.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fiction_union['categories'] = df['categories'].replace(li, 'Fiction')


In [31]:
df_fiction_union

,title,authors,categories,description,average_rating,ratings_count
0,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...,3.85,361.0
1,Spider's Web - A Novel,Charles Osborne;Agatha Christie,detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,3.83,5164.0
2,The One Tree,Stephen R. Donaldson,Fiction,Volume Two of Stephen Donaldson's acclaimed se...,3.97,172.0
3,Rage of angels,Sidney Sheldon,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",3.93,29532.0
4,The Four Loves,Clive Staples Lewis,christian life,Lewis' work on the nature of love divides love...,4.15,33684.0
...,...,...,...,...,...,...
6805,I Am that - Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,philosophy,This collection of the timeless teachings of o...,4.51,104.0
6806,Secrets Of The Heart,Khalil Gibran,mysticism,NaN,4.08,324.0
6807,Fahrenheit 451,Ray Bradbury,book burning,NaN,3.98,5733.0
6808,The Berlin Phenomenology,Georg Wilhelm Friedrich Hegel,history,Since the three volume edition ofHegel's Philo...,0.00,0.0


In [45]:
# BUT THIS CREATES DUPLICATES!!! -> LOTR, genre Fiction and LOTH, genre fictional character
# in the dataset they a re different entries 
#                BUT we can only tell by description once we change the category (both will be fiction)
mask_title = df_fiction_union['title'].str.contains('The Lord of the Rings', case=False)
mask_genre = df_fiction_union['categories'].str.lower().str.contains('fiction')

# Combine the masks using bitwise AND (&) operator
combined_mask = mask_title & mask_genre
df_fiction_union[combined_mask].head(5)
# We can see multiple LOTR entries that are, essentially, the same book but in different editions

,title,authors,categories,description,average_rating,ratings_count
58,The Lord of the Rings,J. R. R. Tolkien,Fiction,Sauron has gathered the Rings of Power - the m...,4.49,187.0
64,The Lord of the Rings,John Ronald Reuel Tolkien,Fiction,"Since it was first published in 1954, 'The Lor...",4.49,680.0
80,The Return of the King - Being the Third Part ...,J. R. R. Tolkien,Fiction,THE GREATEST FANTASY EPIC OF OUR TIME While th...,4.52,253.0
1346,The Treason of Isengard - The History of the L...,John Ronald Reuel Tolkien;Christopher Tolkien,Fiction,Fantasy-roman.,4.16,93.0
1349,The Lord of the Rings,J. R. R. Tolkien,Fiction,This single volume edition of J.R.R Tolkien's ...,4.49,550.0


In [47]:
# Current solution - REMOVE DUPLICATES by Title -> since we don't really care about multiple editions, 
#                                                  just the books as content to read
init_len = len(df_fiction_union)
df_unique = df.drop_duplicates(subset='title', keep='first')
print(init_len)
print(len(df_unique)) # 300 entries removed, not a large hit to database

6810
6547


In [49]:
df_unique = df_unique[['title', 'authors', 'categories', 'description', 'average_rating', 'ratings_count']]
df_unique.to_csv("datasets/Preprocessed_unique_titles.csv")